# Local binary pattern - on artificial data
Came from 2020-09-04_Local_binary_pattern_with_covid
and came from 2020-09-29_Local_binary_pattern_with_covid
and came from 2020-10-11_Local_binary_pattern_with_covid

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
import cv2
import skimage
import tifffile
import os
import pandas as pd

import platform
if platform.system() == 'Windows':
    fileroot = 'X:/'
    print('Loading Windows')
if platform.system() == 'Linux':
    fileroot = '/home/jovyan/'
    print('Loading Linux')
if False:
    fileroot = '/home/ubuntu/Documents/'

import sys
sys.path.append(fileroot+'VU_TEX/Ben_utilities/')
import useful_functions as uf 
import time

Loading Linux


In [2]:
import matplotlib
import matplotlib.patheffects as path_effects
from functools import partial
from skimage.feature import local_binary_pattern
from importlib import reload
from itertools import repeat
from datetime import datetime
from tqdm.notebook import trange, tqdm
from joblib import Parallel, delayed
from numba import njit

In [3]:
reload(uf)

<module 'useful_functions' from '/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/HaralickTextureFinder/useful_functions.py'>

In [4]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [5]:
directory_out_top = '/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/'

In [6]:
directory_in_imgs = '/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-07-05b_image_conversion_and_cropping/'
#imgs = os.listdir(directory_in_imgs)
#if 'Thumbs.db' in imgs:
#    imgs.remove('Thumbs.db')
#imgs.sort(key=partial(uf.getvaluefromstringbest, variable='MS', ending='.', mydtype=int))
imgs = ['V11J11-099__Z4_FO3_01__A1.tif']
#imgs = [img for img in imgs if '.npy' in img]
imgs

['V11J11-099__Z4_FO3_01__A1.tif']

In [7]:
import mahotas, numba, numpy, skimage, scipy
print(mahotas.__version__, numba.__version__, numpy.__version__, skimage.__version__, scipy.__version__)

1.4.13 0.57.0 1.24.3 0.20.0 1.10.1


In [8]:
help(mahotas)

Help on package mahotas:

NAME
    mahotas

DESCRIPTION
    Mahotas
    
    A package for computer vision in Python.
    
    Main Features
    -------------
    
    features
        Compute global and local features (several submodules, include SURF and Haralick features)
    convolve
        Convolution and wavelets
    morph
        Morphological features. Most are available at the mahotas level, include erode(), dilate()...
    watershed
        Seeded watershed implementation
    imread/imsave
        read/write image
    
    Documentation: https://mahotas.readthedocs.io/
    
    Citation:
    
        Coelho, Luis Pedro, 2013. Mahotas: Open source software for scriptable
        computer vision. Journal of Open Research Software, 1:e3, DOI:
        http://dx.doi.org/10.5334/jors.ac

PACKAGE CONTENTS
    _bbox
    _center_of_mass
    _convex
    _convolve
    _distance
    _filters
    _histogram
    _interpolate
    _labeled
    _morph
    _thin
    bbox
    bwperim
    cente

In [9]:
x = mahotas.features.haralick(np.array([[0,0,1],[0,0,2]]), return_mean=True, distance=1)
x, len(x), x.dtype

(array([ 0.390625  ,  1.02083333, -0.06067588,  0.50130208,  0.71458333,
         0.8125    ,  0.984375  ,  1.10457396,  1.56290729,  0.04976852,
         1.10457396, -0.35440278,  0.58923757]),
 13,
 dtype('float64'))

In [10]:
help(mahotas.features.haralick)

Help on function haralick in module mahotas.features.texture:

haralick(f, ignore_zeros=False, preserve_haralick_bug=False, compute_14th_feature=False, return_mean=False, return_mean_ptp=False, use_x_minus_y_variance=False, distance=1)
    feats = haralick(f,
            ignore_zeros=False,
            preserve_haralick_bug=False,
            compute_14th_feature=False,
            return_mean=False,
            return_mean_ptp=False,
            use_x_minus_y_variance=False,
            distance=1
            )
    
    Compute Haralick texture features
    
    Computes the Haralick texture features for the four 2-D directions or
    thirteen 3-D directions (depending on the dimensions of `f`).
    
    ``ignore_zeros`` can be used to have the function ignore any zero-valued
    pixels (as background). If there are no-nonzero neighbour pairs in all
    directions, an exception is raised. Note that this can happen even with
    some non-zero pixels, e.g.::
    
         0 0 0 0
      

In [11]:
#radius_list = [round(1.499*1.327**(float(x))) for x in range(0, 15)]
#print(radius_list); len(radius_list)
distance_list = [1]

In [12]:
channel_list = [0, 1, 2]
print(channel_list)

[0, 1, 2]


In [13]:
#directory_metadata = fileroot + 'VU_TEX/playground_Ben/2022-02-17_UK_CIC/2022-09-06_Iridium_paper_analysis/2022-09-09_merge_metadata_into_iridium_metadata/'
#filename_metadata = '2022-09-12_IMC_and_HnE.xlsx'
#df2 = pd.read_excel(directory_metadata + filename_metadata, engine='openpyxl')
#df2

In [14]:
#df_all = pd.DataFrame(imgs, columns=['Filenames'])
#df_all

In [15]:
#result = df_all.merge(df2, how='outer', indicator=True)
#result[result['_merge'] != 'both']

In [16]:
def get_dims_tiff(filepath):
    myimg = skimage.io.imread(filepath)
    return myimg.shape

df_all = pd.DataFrame(imgs, columns=['Filenames'])
#df_all['dims'] = list(map(uf.get_dims_from_numpy_file, [directory_in_imgs]*len(df_all) + df_all['Filenames']))
df_all['dims'] = list(map(get_dims_tiff, [directory_in_imgs]*len(df_all) + df_all['Filenames']))
#df_all['dims_superpixel'] = list(map(uf.get_cropped_superpixel_dims, df_all['dims'], repeat(1)))

df_all = uf.create_df_cross_for_separate_dfs(df_all,
                                             pd.DataFrame(distance_list, columns=['distance']))

df_all = uf.create_df_cross_for_separate_dfs(df_all,
                                             pd.DataFrame(channel_list , columns=['channel']))

#df_all['n_points'] = [int(32/5*r) for r in df_all['radius']]
df_all['n_points'] = 13
df_all['method'] = ['haralickMean']*len(df_all)

df_all['Subfolder'] = ['Ch_' + str(channel) + '_METHOD_'+ method + '_npoints_' + str(n_points) + '_distance_' + str(distance) + '/' for
                      channel, method, n_points, distance in zip(df_all['channel'],
                                                      df_all['method'], 
                                                       df_all['n_points'],
                                                         df_all['distance'])]

df_all['Fpath_out'] = [directory_out_top + subfolder_out + filename.replace('.tif', '.npy')
                      for subfolder_out, filename
                       in zip(df_all['Subfolder'], df_all['Filenames'])]

df_all['patchsize'] = 242

df_all.sort_values(['distance', 'Filenames'], inplace=True,
                  ascending=True)
df_all.reset_index(drop=True, inplace=True)

df_all

,Filenames,dims,distance,channel,n_points,method,Subfolder,Fpath_out,patchsize
0,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,0,13,haralickMean,Ch_0_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_0_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242
1,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,1,13,haralickMean,Ch_1_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_1_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242
2,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,2,13,haralickMean,Ch_2_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_2_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242


In [17]:
def get_numpy_datatype_unsigned_int(largest_value):
    if largest_value <= 255:
        print('In lower 255')
        value = np.uint8
    elif largest_value <= 65535:
        value = np.uint16
    else:
        value = 0
    return value

mymax = np.max(df_all['distance']); print(mymax)
mydtype = get_numpy_datatype_unsigned_int(mymax); print(mydtype)
df_all['mydtype'] = mydtype
df_all

1
In lower 255
<class 'numpy.uint8'>


,Filenames,dims,distance,channel,n_points,method,Subfolder,Fpath_out,patchsize,mydtype
0,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,0,13,haralickMean,Ch_0_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_0_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>
1,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,1,13,haralickMean,Ch_1_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_1_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>
2,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,2,13,haralickMean,Ch_2_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_2_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>


In [18]:
def subfolder_exists(directory_out_top, sub_directory_out):

    if not os.path.exists(directory_out_top+sub_directory_out):
        os.mkdir(directory_out_top+sub_directory_out)
        
    if os.path.exists(directory_out_top+sub_directory_out):
        return True
    else:
        return False
    
df_all['Subfolder_exists'] = [subfolder_exists(directory_out_top, subfolder) for
                             subfolder in df_all['Subfolder']]

print(np.all(df_all['Subfolder_exists']))
print(df_all['Subfolder_exists'].value_counts())
df_all

True
True    3
Name: Subfolder_exists, dtype: int64


,Filenames,dims,distance,channel,n_points,method,Subfolder,Fpath_out,patchsize,mydtype,Subfolder_exists
0,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,0,13,haralickMean,Ch_0_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_0_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>,True
1,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,1,13,haralickMean,Ch_1_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_1_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>,True
2,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,2,13,haralickMean,Ch_2_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_2_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>,True


In [19]:
def fpath_out_exists(fpath_out):
        
    if os.path.exists(fpath_out):
        return True
    else:
        return False
    
df_all['Fpath_out_exists'] = [fpath_out_exists(fpath_out) for
                             fpath_out in df_all['Fpath_out']]

print(np.all(~df_all['Fpath_out_exists']))
print(df_all['Fpath_out_exists'].value_counts())
df_all

True
False    3
Name: Fpath_out_exists, dtype: int64


,Filenames,dims,distance,channel,n_points,method,Subfolder,Fpath_out,patchsize,mydtype,Subfolder_exists,Fpath_out_exists
0,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,0,13,haralickMean,Ch_0_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_0_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>,True,False
1,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,1,13,haralickMean,Ch_1_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_1_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>,True,False
2,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,2,13,haralickMean,Ch_2_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_2_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>,True,False


In [20]:
df_to_iter = df_all.loc[df_all['Fpath_out_exists'] == False]
df_to_iter

,Filenames,dims,distance,channel,n_points,method,Subfolder,Fpath_out,patchsize,mydtype,Subfolder_exists,Fpath_out_exists
0,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,0,13,haralickMean,Ch_0_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_0_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>,True,False
1,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,1,13,haralickMean,Ch_1_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_1_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>,True,False
2,V11J11-099__Z4_FO3_01__A1.tif,"(28846, 25794, 3)",1,2,13,haralickMean,Ch_2_METHOD_haralickMean_npoints_13_distance_1/,/mnt/hdd/bwoodhams/2023-07-05_Thymus/2023-08-30_haralick/2023-08-31_Haralick_top/Ch_2_METHOD_haralickMean_npoints_13_distance_1/V11J11-099__Z4_FO3_01__A1.npy,242,<class 'numpy.uint8'>,True,False


In [21]:
list_of_images = list(pd.unique(df_to_iter['Filenames']))
print(len(list_of_images))
list_of_images

1


['V11J11-099__Z4_FO3_01__A1.tif']

In [26]:
#@njit
def run_haralick_on_patches(img, patchsize, n_points, distance):
    sizex0 = int(img.shape[0]/patchsize)
    sizex1 = int(img.shape[1]/patchsize)
    output = np.zeros((sizex0, sizex1, n_points), dtype=float)
    for i in trange(sizex0):
        for j in range(sizex1):
            this_patch = img[i*patchsize:(i+1)*patchsize, 
                             j*patchsize:(j+1)*patchsize]
            haralick_values = mahotas.features.haralick(this_patch, return_mean=True, distance=distance)
            output[i,j, 0:len(haralick_values)] = haralick_values
    return output


def apply_Haralick_to_img(img, this_df, i):
    
#    lbp = local_binary_pattern(img[:,:, this_df.iloc[i]['channel']], 
#                               this_df.iloc[i]['n_points'], 
#                               this_df.iloc[i]['radius'], 
#                               this_df.iloc[i]['method']).astype(
#        this_df.iloc[i]['mydtype'])
    
    img = uf.crop_to_superpixel(img, this_df.iloc[i]['patchsize'])
    haralick_values = run_haralick_on_patches(img, 
                                         this_df.iloc[i]['patchsize'], 
                                         this_df.iloc[i]['n_points'],
                                         this_df.iloc[i]['distance'])
    np.save(this_df.iloc[i]['Fpath_out'], haralick_values)
#    print(lbp)
    return 0
#    np.save(this_df.iloc[i]['Fpath_out'], lbp)

def load_images(image_fname, df_all, directory_in_imgs):
    
#    img = np.load(directory_in_imgs + image_fname)
    img = skimage.io.imread(directory_in_imgs + image_fname)
    
    this_df = df_all.loc[df_all['Filenames'] == image_fname]
    
#    apply_LBP_to_img(img, this_df)

    Parallel(n_jobs=3)(delayed(apply_Haralick_to_img)(img, this_df, 
                                                i) 
                   for i in 
                   trange(len(this_df)))
    
    
    
    return 0

In [27]:
#this_fname = df_all.iloc[0]['Filenames']
#this_outside_df = df_all.loc[df_all['Filenames'] == this_fname]
#img = np.load(directory_in_imgs + this_fname)
#lbp = apply_LBP_to_img(img, this_outside_df,
#                      0)
#display(this_outside_df)

In [28]:


#myindices = uf.get_block_indices_without_edges(lbp.shape[0], lbp.shape[1], 45)
#bincount_the_patches(lbp, 45, 7)


In [29]:
Parallel(n_jobs=1)(delayed(load_images)(img_fname, df_all, directory_in_imgs) 
                   for img_fname in 
                   tqdm(list_of_images,
                        total=len(list_of_images)))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[0]

  0%|          | 0/119 [00:00<?, ?it/s]


## 